In [30]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
data = pd.read_csv("daft.csv")

In [32]:
data.head()

,id,title,featuredLevel,publishDate,price,numBedrooms,numBathrooms,propertyType,propertySize,category,...,seller_name,seller_branch,sellerType,m_totalImages,m_hasVideo,m_hasVirtualTour,m_hasBrochure,ber_rating,longitude,latitude
0,3626025,"11 Chestnut Crescent, Bridgemount, Carrigaline...",featured,2022-01-28,290000,3,3,End of Terrace,96.0,Buy,...,Roy Dennehy,Dennehy Auctioneers,BRANDED_AGENT,16.0,False,False,False,C2,-8.382500,51.822940
1,3675175,"58 The Glen, Kilnacourt Woods, Portarlington, ...",featured,2022-01-28,225000,3,2,Semi-D,93.0,Buy,...,Marie Kiernan,Tom McDonald & Associates,BRANDED_AGENT,33.0,False,False,False,C1,-7.177098,53.157465
2,3673450,"16 Dodderbrook Park, Ballycullen, Dublin 24",featured,2022-01-27,575000,4,3,Semi-D,162.0,Buy,...,Moovingo,Moovingo,BRANDED_AGENT,38.0,False,True,False,A3,-6.342763,53.269493
3,3649708,"31 Lissanalta Drive, Dooradoyle, Co. Limerick",featured,2022-01-28,299000,3,3,Semi-D,NaN,Buy,...,DNG Cusack Dunne,DNG Cusack Dunne,BRANDED_AGENT,22.0,False,False,False,C2,-8.640716,52.629588
4,3643947,"5 Columba Terrace, Kells, Co. Meath",featured,2022-01-28,120000,3,1,Terrace,68.0,Buy,...,REA T&J Gavigan,REA T & J Gavigan,BRANDED_AGENT,5.0,False,False,False,G,-6.879797,53.728601


* #### id is not required, we can work with index.
* #### Title is not required.
* #### featuredLevel is not required as it only portrays the Ad on the listings website.
* #### seller_name is not required as seller_branch is available in the data.

In [33]:
data = data.drop(['id', 'title', 'featuredLevel', 'seller_name', 'category', 'AMV_price', 'seller_branch'], axis=1)


### Data Exploration

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3967 entries, 0 to 3966
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   publishDate       3967 non-null   object 
 1   price             3967 non-null   int64  
 2   numBedrooms       3967 non-null   int64  
 3   numBathrooms      3967 non-null   int64  
 4   propertyType      3967 non-null   object 
 5   propertySize      3612 non-null   float64
 6   sellerId          3967 non-null   float64
 7   sellerType        3967 non-null   object 
 8   m_totalImages     3967 non-null   float64
 9   m_hasVideo        3967 non-null   bool   
 10  m_hasVirtualTour  3967 non-null   bool   
 11  m_hasBrochure     3967 non-null   bool   
 12  ber_rating        3967 non-null   object 
 13  longitude         3967 non-null   float64
 14  latitude          3967 non-null   float64
dtypes: bool(3), float64(5), int64(3), object(4)
memory usage: 383.7+ KB


* #### propertySize is missing around 355 values, which is around 10% of the total, hence dropping the records will impact the performance of our model therefore we need to fill the missing values with mean value.
* #### Date is represented by an object, we need to convert this to DateTime

In [35]:
mean_value = data['propertySize'].mean() 
data['propertySize'].fillna(value=mean_value, inplace=True)

In [36]:
data['publishDate'] = pd.to_datetime(data['publishDate'])

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3967 entries, 0 to 3966
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   publishDate       3967 non-null   datetime64[ns]
 1   price             3967 non-null   int64         
 2   numBedrooms       3967 non-null   int64         
 3   numBathrooms      3967 non-null   int64         
 4   propertyType      3967 non-null   object        
 5   propertySize      3967 non-null   float64       
 6   sellerId          3967 non-null   float64       
 7   sellerType        3967 non-null   object        
 8   m_totalImages     3967 non-null   float64       
 9   m_hasVideo        3967 non-null   bool          
 10  m_hasVirtualTour  3967 non-null   bool          
 11  m_hasBrochure     3967 non-null   bool          
 12  ber_rating        3967 non-null   object        
 13  longitude         3967 non-null   float64       
 14  latitude          3967 n

* #### Data is split in the early phase so that we can analyze the performance of our model (which uses train_data).
* #### Test data will remain untouched until the very end.

In [38]:
from sklearn.model_selection import train_test_split

X = data.drop(['price'], axis=1)
y = data['price']

#### https://onlinelibrary.wiley.com/doi/full/10.1002/sam.11583
#### According to this study, the best test to train split ratio is the square root of number of features to 1. We have 18 features which gives us around 4.2:1 , which means the best split is 80/20.

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [40]:
train_data = X_train.join(y_train)

In [41]:
train_data.head()

,publishDate,numBedrooms,numBathrooms,propertyType,propertySize,sellerId,sellerType,m_totalImages,m_hasVideo,m_hasVirtualTour,m_hasBrochure,ber_rating,longitude,latitude,price
291,2022-01-30,4,2,Terrace,149.000000,793.0,UNBRANDED_AGENT,23.0,False,False,False,D1,-6.347112,53.717310,485000
134,2022-01-12,3,3,End of Terrace,106.000000,1250.0,BRANDED_AGENT,15.0,False,False,False,C2,-6.717467,53.748634,315000
1287,2022-01-28,3,3,Semi-D,132.673865,8806.0,BRANDED_AGENT,14.0,False,False,False,C3,-7.718920,52.790241,279000
2377,2022-01-27,4,1,Detached,186.000000,1413.0,BRANDED_AGENT,11.0,False,False,False,B3,-6.331019,53.342291,1200000
3496,2022-01-28,2,1,Apartment,49.000000,8774.0,BRANDED_AGENT,16.0,True,False,False,E1,-9.054026,53.270136,365000


#### We have the training dataset, we can now perform preprocessing techniques such as encoding, standardizaiton and feature engineering etc.

#### Splitting DateTime to Year, Month and Day

In [42]:
train_data['year'] = train_data['publishDate'].dt.year
train_data['month'] = train_data['publishDate'].dt.month
train_data['day'] = train_data['publishDate'].dt.day

train_data.drop(['publishDate'], axis=1, inplace = True)

#### Converting bool to int

In [43]:
train_data.replace({False: 0, True: 1}, inplace=True)

#### Dropping missing energy rating values as these cannot be replaced.

In [44]:
train_data['ber_rating'].value_counts().get('XXX', 0)

134

In [45]:
train_data.drop(train_data[train_data['ber_rating'] == 'XXX'].index, inplace=True)

#### Dealing with High Cardinality in Categorical Features
##### We can see that the amount of unique values is alot for using One Hot Encoding as it will introduce Curse of Dimensionality.
##### Hence, we use simple aggregation to convert all values to 'other' except the 3 most occuring values and then we can use Label Encoding.

In [46]:
train_data['propertyType'].value_counts()

propertyType
Detached          813
Semi-D            751
Apartment         562
Terrace           471
End of Terrace    185
Bungalow          154
Townhouse          48
Duplex             40
Site               12
House               2
Studio              1
Name: count, dtype: int64

In [47]:
train_data['propertyType'].replace(['Terrace', 'End of Terrace', 'Bungalow', 'Townhouse', 'Duplex', 'Site', 'House', 'Studio'], 'Other', inplace=True)

In [48]:
train_data['propertyType'].value_counts()

propertyType
Other        913
Detached     813
Semi-D       751
Apartment    562
Name: count, dtype: int64

In [49]:
from sklearn import preprocessing 

label_encoder = preprocessing.LabelEncoder() 

train_data['propertyType'] = label_encoder.fit_transform(train_data['propertyType'])
train_data['sellerType'] = label_encoder.fit_transform(train_data['sellerType'])

In [50]:
train_data.head()

,numBedrooms,numBathrooms,propertyType,propertySize,sellerId,sellerType,m_totalImages,m_hasVideo,m_hasVirtualTour,m_hasBrochure,ber_rating,longitude,latitude,price,year,month,day
291,4,2,2,149.000000,793.0,2,23.0,0,0,0,D1,-6.347112,53.717310,485000,2022,1,30
134,3,3,2,106.000000,1250.0,0,15.0,0,0,0,C2,-6.717467,53.748634,315000,2022,1,12
1287,3,3,3,132.673865,8806.0,0,14.0,0,0,0,C3,-7.718920,52.790241,279000,2022,1,28
2377,4,1,1,186.000000,1413.0,0,11.0,0,0,0,B3,-6.331019,53.342291,1200000,2022,1,27
3496,2,1,0,49.000000,8774.0,0,16.0,1,0,0,E1,-9.054026,53.270136,365000,2022,1,28


#### Encoding Energy Ratings

In [51]:
train_data['ber_rating'].value_counts()

ber_rating
C2        395
C3        376
C1        357
D1        355
D2        290
B3        224
G         193
E1        175
SI_666    144
E2        136
F         127
A2         83
B2         83
A3         77
B1         19
A1          4
A1A2        1
Name: count, dtype: int64

In [52]:
train_data = train_data[train_data.ber_rating != 'A1A2']

In [53]:
train_data['ber_rating'].value_counts()

ber_rating
C2        395
C3        376
C1        357
D1        355
D2        290
B3        224
G         193
E1        175
SI_666    144
E2        136
F         127
B2         83
A2         83
A3         77
B1         19
A1          4
Name: count, dtype: int64

In [54]:
train_data['ber_rating'].replace({'A1': 'high', 'A2': 'high', 'A3': 'high', 
                                   'B1': 'med', 'B2': 'med', 'B3': 'med', 
                                   'C1': 'low', 'C2': 'low', 'C3': 'low', 
                                   'D1': 'low', 'D2': 'low', 'D3': 'low', 
                                   'E1': 'low', 'E2': 'low', 'F': 'low', 
                                   'G': 'low', 'SI_666': 'low'}, inplace=True)

In [55]:
train_data['ber_rating'].value_counts()

ber_rating
low     2548
med      326
high     164
Name: count, dtype: int64

In [58]:
train_data['ber_rating'] = label_encoder.fit_transform(train_data['ber_rating'])

In [59]:
train_data.head()

,numBedrooms,numBathrooms,propertyType,propertySize,sellerId,sellerType,m_totalImages,m_hasVideo,m_hasVirtualTour,m_hasBrochure,ber_rating,longitude,latitude,price,year,month,day
291,4,2,2,149.000000,793.0,2,23.0,0,0,0,1,-6.347112,53.717310,485000,2022,1,30
134,3,3,2,106.000000,1250.0,0,15.0,0,0,0,1,-6.717467,53.748634,315000,2022,1,12
1287,3,3,3,132.673865,8806.0,0,14.0,0,0,0,1,-7.718920,52.790241,279000,2022,1,28
2377,4,1,1,186.000000,1413.0,0,11.0,0,0,0,2,-6.331019,53.342291,1200000,2022,1,27
3496,2,1,0,49.000000,8774.0,0,16.0,1,0,0,1,-9.054026,53.270136,365000,2022,1,28


In [60]:
train_data['ber_rating'].value_counts()

ber_rating
1    2548
2     326
0     164
Name: count, dtype: int64